In [50]:
from lightgbm import LGBMRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, RobustScaler, StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, LassoCV, LinearRegression
import time
from datetime import datetime
import matplotlib.pyplot as plt
import os
import numpy.ma as ma
from sklearn import base
import warnings
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [51]:
path_train = "data/train_data.csv"
path_test = "data/test_data.csv"
target_ = "contest-tmp2m-14d__tmp2m"

In [52]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

date_col = "startdate"

df_train[date_col] = pd.to_datetime(df_train[date_col])
df_test[date_col] = pd.to_datetime(df_test[date_col])

In [53]:
df_train['wind-hgt-250-2010-10']

KeyError: 'wind-hgt-250-2010-10'

In [36]:
def filter_na_cols(df):
    count_na_df = df.isna().sum()
    if count_na_df[count_na_df > 0].tolist():
        return count_na_df[count_na_df > 0]
    else:
        return 'Clean dataset'

col_na = [
    'nmme0-tmp2m-34w__ccsm30',
    'nmme-tmp2m-56w__ccsm3',
    'nmme-prate-34w__ccsm3',
    'nmme0-prate-56w__ccsm30',
    'nmme0-prate-34w__ccsm30',
    'nmme-prate-56w__ccsm3',
    'nmme-tmp2m-34w__ccsm3']

g_means =  ['nmme0-tmp2m-34w__nmme0mean',
 'nmme-tmp2m-56w__nmmemean',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-56w__nmme0mean',
 'nmme0-prate-34w__nmme0mean',
 'nmme-prate-56w__nmmemean',
 'nmme-tmp2m-34w__nmmemean']


g_1 = ['nmme0-tmp2m-34w__cancm30',
'nmme0-tmp2m-34w__cancm40',
'nmme0-tmp2m-34w__ccsm40',
'nmme0-tmp2m-34w__cfsv20',
'nmme0-tmp2m-34w__gfdlflora0',
'nmme0-tmp2m-34w__gfdlflorb0',
'nmme0-tmp2m-34w__gfdl0',
'nmme0-tmp2m-34w__nasa0']

g_2 = ['nmme-tmp2m-56w__cancm3',
'nmme-tmp2m-56w__cancm4',
'nmme-tmp2m-56w__ccsm4',
'nmme-tmp2m-56w__cfsv2',
'nmme-tmp2m-56w__gfdl',
'nmme-tmp2m-56w__gfdlflora',
'nmme-tmp2m-56w__gfdlflorb',
'nmme-tmp2m-56w__nasa']

g_3 = ['nmme-prate-34w__cancm3',
'nmme-prate-34w__cancm4',
'nmme-prate-34w__ccsm4',
'nmme-prate-34w__cfsv2',
'nmme-prate-34w__gfdl',
'nmme-prate-34w__gfdlflora',
'nmme-prate-34w__gfdlflorb',
'nmme-prate-34w__nasa']

g_4 = [ 'nmme0-prate-56w__cancm30',
'nmme0-prate-56w__cancm40',
'nmme0-prate-56w__ccsm40',
'nmme0-prate-56w__cfsv20',
'nmme0-prate-56w__gfdlflora0',
'nmme0-prate-56w__gfdlflorb0',
'nmme0-prate-56w__gfdl0',
'nmme0-prate-56w__nasa0']

g_5 = ['nmme0-prate-34w__cancm30',
'nmme0-prate-34w__cancm40',
'nmme0-prate-34w__ccsm40',
'nmme0-prate-34w__cfsv20',
'nmme0-prate-34w__gfdlflora0',
'nmme0-prate-34w__gfdlflorb0',
'nmme0-prate-34w__gfdl0',
'nmme0-prate-34w__nasa0']

g_6 = ['nmme-prate-56w__cancm3',
'nmme-prate-56w__cancm4',
'nmme-prate-56w__ccsm4',
'nmme-prate-56w__cfsv2',
'nmme-prate-56w__gfdl',
'nmme-prate-56w__gfdlflora',
'nmme-prate-56w__gfdlflorb',
'nmme-prate-56w__nasa']

g_7 = ['nmme-tmp2m-34w__cancm3',
'nmme-tmp2m-34w__cancm4',
'nmme-tmp2m-34w__ccsm4',
'nmme-tmp2m-34w__cfsv2',
'nmme-tmp2m-34w__gfdl',
'nmme-tmp2m-34w__gfdlflora',
'nmme-tmp2m-34w__gfdlflorb',
'nmme-tmp2m-34w__nasa']

gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7]

zip_cols = zip(col_na, gs, g_means)
for c, g, m in tqdm(zip_cols):
    df_train[c] = (df_train[m]*9) - df_train[g].sum(1)

7it [00:03,  2.26it/s]


In [37]:
df_train.drop(columns='ccsm30', inplace=True)
df_test.drop(columns='ccsm30', inplace=True)

In [38]:
target_ = 'contest-tmp2m-14d__tmp2m'

col = [each for each in df_train.columns if "contest" in each and each != target_]

def get_idx(lat, lon):
    return str(round(lat, 4)) + "_" + str(round(lon, 4))

df_train['idx'] = np.vectorize(get_idx)(df_train['lat'], df_train['lon'])
df_test['idx'] = np.vectorize(get_idx)(df_test['lat'], df_test['lon'])

for each in col:
    df_train[each + "_lag_1"] = df_train.groupby("idx")[each].shift(1).bfill()
    df_test[each + "_lag_1"] = df_test.groupby("idx")[each].shift(1).bfill()

    df_train[each + "_shift_1"] = df_train.groupby("idx")[each].shift(-1).ffill()
    df_test[each + "_shift_1"] = df_test.groupby("idx")[each].shift(-1).ffill()

    df_train[each + "_lag_2"] = df_train.groupby("idx")[each].shift(2).bfill()
    df_test[each + "_lag_2"] = df_test.groupby("idx")[each].shift(2).bfill()

    df_train[each + "_shift_2"] = df_train.groupby("idx")[each].shift(-2).ffill()
    df_test[each + "_shift_2"] = df_test.groupby("idx")[each].shift(-2).ffill()

In [39]:
imp_contest_cols = [
    'contest-pevpr-sfc-gauss-14d__pevpr',
    'contest-pres-sfc-gauss-14d__pres',
    'contest-prwtr-eatm-14d__prwtr',
    'contest-slp-14d__slp',
    'contest-wind-h10-14d__wind-hgt-10',
    'contest-wind-h100-14d__wind-hgt-100',
    'contest-wind-h500-14d__wind-hgt-500',
    'contest-wind-uwnd-250-14d__wind-uwnd-250',
    'contest-wind-vwnd-925-14d__wind-vwnd-925'
]

def get_name_cols(suffix=''):
    name_cols = [col + f'{suffix}' for col in imp_contest_cols]

    return name_cols

def feature_engineer(df, cols):

    # df.loc[:, 'idx'] = np.vectorize(get_idx)(df['lat'], df['lon'])
    df.loc[:, 'month'] = df['startdate'].dt.month
    df.loc[:, 'year']  = df['startdate'].dt.year

    mean_contest_cols = df.groupby(['idx', 'month', 'year'])[cols].transform(lambda x: x.mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_mean'))))
    std_contest_cols = df.groupby(['idx', 'month', 'year'])[cols].transform(lambda x: x.std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_std'))))
    rolling_mean_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=7, min_periods=3, win_type="triang").mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_mean'))))
    rolling_std_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=7, min_periods=3).std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_std'))))
    expanding_mean_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.expanding(2).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_expanding_mean'))))
    expanding_std_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.expanding(2).std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_expanding_std'))))
    exponentially_weighted_average = df.groupby('idx')[cols].transform(lambda x: x.shift(1).ewm(alpha=0.95).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_ewa'))))

    df = pd.concat([df, mean_contest_cols, std_contest_cols, rolling_mean_contest_cols,
                    rolling_std_contest_cols, expanding_mean_contest_cols,
                    expanding_std_contest_cols, exponentially_weighted_average], axis=1)

    df = df.fillna(method='bfill')
    df.drop(columns=['month', 'year'], inplace=True)

    return df

df_train = feature_engineer(df_train, imp_contest_cols)
df_test = feature_engineer(df_test, imp_contest_cols)

In [40]:
df_train.drop(columns=[target_, 'index'], inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(50, 50))
ax = plt.plot(df_train['startdate'], df_train['contest-precip-14d__precip'])